## Обход Википедии

Данные обхода представлены в файле формата json, прилагаемом к notebook. Также прилагаю код получения обхода (в архиве).

## Построение графа


Читаем из json файла данные обхода веб-графа и формируем ориентированный граф.

In [1]:
import networkx as nx
from pprint import pprint

In [2]:
import json

with open("graph.json") as f:
    data = json.load(f)
    
edges = [(u["url"], v) for u in data for v in u["links"] ]
graph = nx.DiGraph()
graph.add_edges_from(edges)


### Некоторые характеристики графа:

In [3]:
print "{} вершин и {} ребер".format(graph.order(), graph.size())
print "плотность графа: {}".format(nx.density(graph))

647753 вершин и 2079691 ребер
плотность графа: 4.95656284169e-06


Таким образом, веб-граф является разреженным.

## Вычисление PageRank для вершин графа при различных $\alpha$:

In [4]:
import copy
import operator
TOP_PAGES_COUNT = 10
alphas = [0.85, 0.95, 0.5, 0.3]
alphas_top_pages = []
pr_alphas = []


def print_page_info(page, page_property):
    print u"{0} {1:0.8f} \n{2} \n{3}".format(page["title"], page[page_property], page["url"], page["snippet"])

In [5]:


for alpha in alphas:
    pr = nx.pagerank(graph, alpha=alpha)
    
    pr_alphas.append(pr)
    
    top_pages_temp = sorted(pr.items(), key=operator.itemgetter(1), reverse=True)[:TOP_PAGES_COUNT]
    top_pages = []
    
    for tpt in top_pages_temp:
        if tpt in data:
            tpt_copy = copy.copy(tpt)
            tpt_copy["page_rank"] = pr[tpt_copy["url"]]

            top_pages.append(tpt_copy)
        else:
            tpt_new = {}
            tpt_new["page_rank"] = pr[tpt[0]]
            tpt_new["snippet"] = tpt_new["title"] = "Empty"
            tpt_new["url"] = tpt[0]
            top_pages.append(tpt_new)
    
    alphas_top_pages.append(top_pages)

    print "\n\n-----alpha={}-----\n\n".format(alpha)
    for page in top_pages:
        print_page_info(page, "page_rank")
        print
    

                         
                         



-----alpha=0.85-----


Empty 0.00004778 
https://en.wikipedia.org/wiki/Geographic_coordinate_system 
Empty

Empty 0.00004582 
https://en.wikipedia.org/wiki/International_Standard_Book_Number 
Empty

Empty 0.00002475 
https://en.wikipedia.org/wiki/United_States 
Empty

Empty 0.00002397 
https://en.wikipedia.org/wiki/Virtual_International_Authority_File 
Empty

Empty 0.00001927 
https://en.wikipedia.org/wiki/Library_of_Congress_Control_Number 
Empty

Empty 0.00001754 
https://en.wikipedia.org/wiki/Wayback_Machine 
Empty

Empty 0.00001645 
https://en.wikipedia.org/wiki/Integrated_Authority_File 
Empty

Empty 0.00001610 
https://en.wikipedia.org/wiki/Digital_object_identifier 
Empty

Empty 0.00001600 
https://en.wikipedia.org/wiki/IMDb 
Empty

Empty 0.00001484 
https://en.wikipedia.org/wiki/International_Standard_Name_Identifier 
Empty



-----alpha=0.95-----


Empty 0.00005322 
https://en.wikipedia.org/wiki/Geographic_coordinate_system 
Empty

Empty 0.00005102 
https://en.wikipedia.org/

#### В более сжатом виде (только URL и PageRank):

In [6]:
for i, top_pages in enumerate(alphas_top_pages):
    print "\n\n-----alpha={}-----\n\n".format(alphas[i])
    pprint([(t["url"], t["page_rank"]) for t in top_pages])



-----alpha=0.85-----


[(u'https://en.wikipedia.org/wiki/Geographic_coordinate_system',
  4.778009922520809e-05),
 (u'https://en.wikipedia.org/wiki/International_Standard_Book_Number',
  4.5815952238721774e-05),
 (u'https://en.wikipedia.org/wiki/United_States', 2.4746442450869328e-05),
 (u'https://en.wikipedia.org/wiki/Virtual_International_Authority_File',
  2.39657666191829e-05),
 (u'https://en.wikipedia.org/wiki/Library_of_Congress_Control_Number',
  1.9271364360578078e-05),
 (u'https://en.wikipedia.org/wiki/Wayback_Machine', 1.7539264628181394e-05),
 (u'https://en.wikipedia.org/wiki/Integrated_Authority_File',
  1.6451960522853294e-05),
 (u'https://en.wikipedia.org/wiki/Digital_object_identifier',
  1.6098729839234703e-05),
 (u'https://en.wikipedia.org/wiki/IMDb', 1.5998731349018203e-05),
 (u'https://en.wikipedia.org/wiki/International_Standard_Name_Identifier',
  1.4836295469592081e-05)]


-----alpha=0.95-----


[(u'https://en.wikipedia.org/wiki/Geographic_coordinate_system',
  

### Анализ:

Исходными ссылками были: "https://en.wikipedia.org/wiki/Ludwig_van_Beethoven",
        "https://en.wikipedia.org/wiki/Franz_Schubert",
        "https://en.wikipedia.org/wiki/Felix_Mendelssohn",
        "https://en.wikipedia.org/wiki/Franz_Liszt",
        "https://en.wikipedia.org/wiki/Robert_Schumann"
        
В результате в топе оказались страницы со значительно отличающейся тематикой. В целом влияние параметра $\alpha$ на данном примере не видно. Видим зависимость PageRank от $\alpha$: чем больше $\alpha$, тем больше значения PageRank.

Отсутствие тайтла, сниппета объясняется, возможно, тем, что spider во время обхода не попал на эти страницы (потому что я остановил обход на 10177 странице)


In [7]:
auth_hits, hub_hits = nx.hits(graph)

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [ ]:
avg_hits = dict()
for auth in auth_hits:
    url = auth[0]
    auth_value = auth[1]
    hub_value = hub_hits[url][1]
    avg_hits[url] = (auth_value + hub_value) / 2
    
top_pages_auth_hits = sorted(auth_hits.items(), key=operator.itemgetter(1), reverse=True)[:TOP_PAGES_COUNT]
top_pages_hubs_hits = sorted(hub_hits.items(), key=operator.itemgetter(1), reverse=True)[:TOP_PAGES_COUNT]
top_pages_avg_hits = sorted(avg_hits.items(), key=operator.itemgetter(1), reverse=True)[:TOP_PAGES_COUNT]

print "\n\n-----auth-----\n\n"
pprint(top_pages_auth_hits)

print "\n\n-----hub-----\n\n"
pprint(top_pages_hubs_hits)

print "\n\n-----avg-----\n\n"
pprint(top_pages_avg_hits)


### Анализ:

К сожалению, алгоритм не сошелся на таком количестве итераций.